In [8]:
tempsPath = '/user/s292129/data/exam_ex2_data/Temperatures.txt'

#Schema :  date_reading,city,country,max_temperature,min_temperature 


# Task A – Meteorological summer 2015 – City average maximum temperature
tempsRDD = sc.textFile(tempsPath)
summerTempsRDD = tempsRDD.filter( lambda line: line.split(',')[0].startswith('2015/06')\
                                 or line.split(',')[0].startswith('2015/07') or line.split(',')[0].startswith('2015/08'))
def mapFunc1(line):
    fields = line.split(',');
    city = fields[1]
    country = fields[2]
    maxTemp = float(fields[3])
    return (city+'-'+country, (maxTemp, +1))

cityCountry_sumsRDD  = summerTempsRDD.map(mapFunc1).reduceByKey(lambda v1,v2: (v1[0]+v2[0], v1[1]+v2[1])) 
cityCountry_averageRDD  = cityCountry_sumsRDD.mapValues( lambda v: v[0]/v[1]).cache()
cityCountry_averageRDD.collect()



[('Rome-Italy', 22.5), ('Turin-Italy', 33.166666666666664)]

In [9]:
# Task B – Meteorological summer 2015 – Hot cities
countryMappedRDD = cityCountry_averageRDD.map(lambda pair: (pair[0].split('-')[1], (pair[1], 1)))
country_sumsRDD = countryMappedRDD.reduceByKey( lambda v1, v2: (v1[0]+v2[0], v1[1]+v2[1]))
country_averageRDD  = country_sumsRDD.mapValues(lambda v : v[0]/v[1])
country_cityAvgRDD = cityCountry_averageRDD.map(lambda pair: (pair[0].split('-')[1], (pair[0].split('-')[0], pair[1])))
country__cityAvg_countryAvg = country_cityAvgRDD.join(country_averageRDD)
country__cityAvg_countryAvg.values().filter(lambda pair:  pair[0][1] > (pair[1] + 5)).keys().keys().collect()


['Turin']